# Graph API

In this notebook we'll work to achieve milestone 1: the ability to read my personal OneNote files which are stored in my personal OneDrive account in Azure.

In [165]:
from dataclasses import dataclass
import requests
from typing import List

from azure.identity import DeviceCodeCredential, InteractiveBrowserCredential
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from msal import ConfidentialClientApplication

from danu.infra import LocalJsonSecretsProvider

## Create App Registration and Authentication

The first step is to create an Azure App Registration.  The reason we do this is it gives our application an identity in Azure which we can use to grant permissions.  We do this in the Azure Portal.  I named mine "Danu Agent".

If this were a commercial app, I'd be using managed identity.  But for now for local experimentation I'm going to create a client secret and make a way to retrieve these credentials.

## Create config file

Create a directory `config` in the project root, and create `config/danu-config.json` there as shown below.  Note that this file is not in source control and will not be checked in:

`config/danu-config.json`:
```json
{
    "danu-identity":
    {
        "app_id": "app id goes here. Fill this in from your app registration you created.",
        "client_secret": "app secret goes here. Create a client secret and put it here."
    }
}
```

In [2]:
secrets = LocalJsonSecretsProvider('../../../config/danu-config.json') # This is at the top level, same level as 'src'

In [3]:
app_id = secrets.get_secret('danu-identity.app_id')
client_secret = secrets.get_secret('danu-identity.client_secret')
tenant_id = secrets.get_secret('danu-identity.tenant_id')
print(f'app id {app_id} tenant={tenant_id} secret len={len(client_secret)}')

app id 52e677b3-b988-4573-b9d0-18cdb5c7c67b tenant=32dd5da8-160d-4bc8-9fcc-9c8ed28043ea secret len=40


We now have an identity for our application established.

## Graph authentication

Working through ideas in [Microsoft Graph authentication and authorization overview](https://learn.microsoft.com/en-us/graph/auth/).


In [5]:
authority = f'https://login.microsoftonline.com/{tenant_id}'
scopes = ['https://graph.microsoft.com/.default']

app = ConfidentialClientApplication(app_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes)

In [6]:
result['token_type'], result['access_token'][0:3] + '...'

('Bearer', 'eyJ...')

In [7]:
access_token = result['access_token']

In [8]:
# Use Graph API to access OneNote data
import requests
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get('https://graph.microsoft.com/v1.0/me/onenote/notebooks', headers=headers)

In [9]:
response, response.content

(<Response [400]>,
 b'{"error":{"code":"BadRequest","message":"/me request is only valid with delegated authentication flow.","innerError":{"date":"2023-10-14T23:35:39","request-id":"727c30e6-0ba0-46c5-960a-b8afa064b00c","client-request-id":"727c30e6-0ba0-46c5-960a-b8afa064b00c"}}}')

One way to handle this would be to use a web site that has a public endpoint for retrieving the credentials via a redirect.  But since we're in a notebook, let's use an `InteractiveBrowserCredential` from the `azure.identity` package.  This will open a browser window for us to authenticate and grant permissions to the application.

In [14]:
redirect_uri = 'http://localhost:59513' # Must add this in portal for app/authentication/platform configurations/redirect uris
credential = InteractiveBrowserCredential(client_id=app_id, tenant_id=tenant_id, redirect_uri=redirect_uri, client_secret=client_secret)
token = credential.get_token('https://graph.microsoft.com/.default', client_secret=client_secret)
access_token = token.token

InteractiveBrowserCredential.get_token failed: AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'.
Trace ID: 06c78d72-316c-4851-988a-c9453e4d1f00
Correlation ID: 6c854f8e-f515-469b-a4a8-ca6e822ed33a
Timestamp: 2023-10-14 23:41:33Z


ClientAuthenticationError: AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'.
Trace ID: 06c78d72-316c-4851-988a-c9453e4d1f00
Correlation ID: 6c854f8e-f515-469b-a4a8-ca6e822ed33a
Timestamp: 2023-10-14 23:41:33Z

I apologize for the confusion earlier. It seems that the InteractiveBrowserCredential does not support the use of a client secret. In this case, you can use the DeviceCodeCredential instead, which is designed for situations where you don't have a web server to receive the credentials.

Note that you must go into your application under Authentication/Advanced and enable **Allow public client flows**.

In [34]:
scope = 'User.Read,Notes.Read'
one_note_content = '/me/onenote/notebooks' # For a OneDrive for business account. See https://learn.microsoft.com/en-us/graph/integrate-with-onenote
#one_note_content = '/me/drive/root/onenote/notebooks' # For a personal OneDrive account
credential = DeviceCodeCredential(client_id=app_id, tenant_id=tenant_id, scope=scope)
token = credential.get_token(scope)
access_token = token.token

headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get('https://graph.microsoft.com/v1.0' + one_note_content, headers=headers)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E6U67WR78 to authenticate.


In [35]:
response.status_code, response.content

(404,
 b'{"error":{"code":"30108","message":"OneDrive for Business for this user account cannot be retrieved.","innerError":{"date":"2023-10-15T01:02:14","request-id":"33bed514-656d-4e7c-b77f-f0df481dc69b","client-request-id":"33bed514-656d-4e7c-b77f-f0df481dc69b"}}}')

In [36]:
access_token='...'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get('https://graph.microsoft.com/v1.0' + one_note_content, headers=headers)

In [37]:
response.status_code, response.content

(200,
 b'{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users(\'richardlack%40hotmail.com\')/onenote/notebooks","value":[{"id":"0-D837C5F832A7D046!513","self":"https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513","createdDateTime":"2016-04-28T15:23:06.233Z","displayName":"Argon Collab","lastModifiedDateTime":"2017-11-04T16:33:20.203Z","isDefault":false,"userRole":"Owner","isShared":true,"sectionsUrl":"https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513/sections","sectionGroupsUrl":"https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513/sectionGroups","createdBy":{"user":{"id":"D837C5F832A7D046","displayName":"Richard Lack"}},"lastModifiedBy":{"user":{"id":"D837C5F832A7D046","displayName":"Richard Lack"}},"links":{"oneNoteClientUrl":{"href":"onenote:https://d.docs.live.net/d837c5f832a7d046/Documents/Argon%20Collab"},"oneN

---

The redirect URI 'http://localhost:59436' specified in the request does not match the redirect URIs configured for the application '52e677b3-b988-4573-b9d0-18cdb5c7c67b'. Make sure the redirect URI sent in the request matches one added to your application in the Azure portal. Navigate to https://aka.ms/redirectUriMismatchError to learn more about how to fix this.

---

We will work to achieve **delegated access** or access on behalf of a user.

In [ ]:
app = ConfidentialClientApplication(app_id, authority='https://login.microsoftonline.com/common',
                                    client_credential=client_secret)

In [ ]:
accounts = app.get_accounts() # Check the cache to see if we have some accounts

In [ ]:
accounts

In [ ]:
app.

For some reason, acquire_token_iteractive isn't available there.  Let's try it with a public application instead of a confidential one.

In [ ]:
from msal import PublicClientApplication

In [ ]:
app = PublicClientApplication(app_id, authority='https://login.microsoftonline.com/consumers')

In [ ]:
result = app.acquire_token_interactive(scopes=['User.Read'])

Result was:
> AADSTS700016: Application with identifier '52e677b3-b988-4573-b9d0-18cdb5c7c67b' was not found in the directory 'Microsoft'. This can happen if the application has not been installed by the administrator of the tenant or consented to by any user in the tenant. You may have sent your authentication request to the wrong tenant.

I got that before, then I switched the authority to be `/consumers` instead of `/common`.  Then I re-ran the code and got this error:

> unauthorized_client: The client does not exist or is not enabled for consumers. If you are the application developer, configure a new application through the App Registrations in the Azure Portal at https://go.microsoft.com/fwlink/?linkid=2083908.

---

Graph API explorer, this is cool [Graph API Explorer](https://developer.microsoft.com/en-us/graph/graph-explorer)

## Temporary Token
I'm having a lot of trouble making this API work,  but the problem is somewhere in how I'm getting my token. I know this because I am able to successfully use [Graph API Explorer](https://developer.microsoft.com/en-us/graph/graph-explorer) to explore my OneNote data, even without having a Sharepoint Online license.  So I'm going to work around this by using the token I got from Graph API Explorer for now, and come back later and figure out how to properly get the token.  I got this token using graph explorer, authorized it for Note.Read, and we're off to the races, hopefully.

In [124]:
access_token = secrets.get_secret('danu-identity.temporary_token')

## OneNote Provider

I can see by exploring Graph explorer that working with this data is going to be an adventure.  To make accessing this easier, let's make our own abstraction for working with OneNote.

In [95]:
@dataclass
class Notebook:
    name: str
    url: str

@dataclass
class NotebookSection:
    name: str
    url: str

@dataclass
class NotebookPageRef:
    """A reference to a notebook page including its title.  Does not include the content."""
    name: str
    url: str # URL for this page entity.
    content_url: str # URL for the content inside of the page.


In [169]:
class OneNoteClient:
    def __init__(self, access_token: str):
        self._token = access_token
    
    def list_notebooks(self) -> List[Notebook]:
        notebooks = []
        resp = self._request('/me/onenote/notebooks')
        value_list = resp['value']
        for notebook_json in value_list:
            url = notebook_json['self']
            name = notebook_json['displayName']
            notebook = Notebook(name=name, url=url)
            notebooks.append(notebook)
        return notebooks
    
    def list_sections(self, notebook: Notebook) -> List[NotebookSection]:
        sections = []
        resp = self._request(notebook.url + '/sections?$select=self,displayName&$orderby=displayName')
        for section_json in resp['value']:
            section = NotebookSection(name=section_json['displayName'], url=section_json['self'])
            sections.append(section)
        return sections

    def list_pages_in_section(self, section: NotebookSection) -> List[NotebookPageRef]:
        resp = self._request(section.url + '/pages?$select=self,title,contenturl')
        pages = []
        # If the data is large we will get a chunk of data with a link for the next chunk.
        while True: # Loop across the chunks of data.
            for page_json in resp['value']:
                page = NotebookPageRef(name=page_json['title'], url=page_json['self'], content_url=page_json['contentUrl'])
                pages.append(page)
            next_link = resp.get('@odata.nextLink')
            
            if not next_link:
                break
            resp = self._request(next_link) # Get the next chunk of data.
        return pages

    def get_page_text(self, page: NotebookPageRef) -> str:
        content = self._request_plan(page.content_url)
        soup = BeautifulSoup(content, features='html.parser')
        return soup.get_text()

    def get_page_html(self, page: NotebookPageRef) -> str:
        return self._request_plain(page.content_url)
    
    def get_page_md(self, page: NotebookPageRef) -> str:
        html_text = self.get_page_html(page)
        return md(html_text)
    
    def find_page_from_path(self, notebook_name: str, section_name: str, page_name: str) -> NotebookPageRef:
        # We could use filters to do this more efficiently. Also there is no error handling.
        notebooks = self.list_notebooks()
        notebook = [n for n in notebooks if n.name == notebook_name][0]
        
        sections = self.list_sections(notebook)
        section = [s for s in sections if s.name == section_name][0]

        pages = self.list_pages_in_section(section)
        page = [p for p in pages if p.name == page_name][0]
        return page


    def _request(self, url):
        """Do an API request and return the result as json."""
        response = self._request_plain(url)
        return response.json()
    
    def _request_plain(self, url):
        """Do a request including the headers, returning the response object."""
        headers = {'Authorization': f'Bearer {self._token}'}
        if url.startswith('https://'):
            request_url = url
        else:
            request_url = 'https://graph.microsoft.com/v1.0' + url

        response = requests.get(request_url, headers=headers)
        if response.status_code != 200:
            raise Exception(f'OneNoteClient got {response.status_code} {response.content}')
        return response.content


one_note = OneNoteClient(access_token)

In [91]:
notebooks = one_note.list_notebooks()
notebooks

[Notebook(name='Argon Collab', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513'),
 Notebook(name='Casa de Lack', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!213'),
 Notebook(name='Edge4x4-TABLET-T4MQ0ET0', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!413'),
 Notebook(name='GuildWebsite', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!11498'),
 Notebook(name='Kuddles', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-9FA50C10908E5E6C!4057'),
 Notebook(name='Lubritech', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!64100'),
 Notebook(name='Mossms', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!196'

In [111]:
notebook = [n for n in notebooks if n.name=="Richard's Notebook"][0]
notebook

Notebook(name="Richard's Notebook", url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!125')

In [115]:
sections = one_note.list_sections(notebook)
sections

[NotebookSection(name='AI', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!10834'),
 NotebookSection(name='Creative', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!210'),
 NotebookSection(name='History', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!10715'),
 NotebookSection(name='Library', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!220'),
 NotebookSection(name='Planning', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!203'),
 NotebookSection(name='Projects', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!771'),
 NotebookSection(name='Quick Notes', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832

In [116]:
section = [s for s in sections if s.name=='Creative'][0]
section

NotebookSection(name='Creative', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!210')

In [117]:
pages = one_note.list_pages_in_section(section)
pages

[NotebookPageRef(name='Story Components', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-958605dfd18b40b68991e0bcaf18e2c1!1-D837C5F832A7D046!210', content_url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-958605dfd18b40b68991e0bcaf18e2c1!1-D837C5F832A7D046!210/content'),
 NotebookPageRef(name='T shirt designs', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-216c60e40c141b0e2c336a6f5fb15316!1-D837C5F832A7D046!210', content_url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-216c60e40c141b0e2c336a6f5fb15316!1-D837C5F832A7D046!210/content'),
 NotebookPageRef(name='Cool names', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-0bb82fed474515092907e2f4a377ee97!1-D837C5F832A7D046!210', content_url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-0bb82fed474515092907e2f4a377ee97!1-D837C5F832

In [118]:
page = [p for p in pages if p.name=='Game Mechanics/ideas'][0]
page

NotebookPageRef(name='Game Mechanics/ideas', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-48eeeebc65d70a4c006c70db1dd24d51!1-D837C5F832A7D046!210', content_url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-48eeeebc65d70a4c006c70db1dd24d51!1-D837C5F832A7D046!210/content')

Notes on working with notebooks in graphAPI while I figure this out.

https://graph.microsoft.com/v1.0/me/onenote/notebooks?$select=id,self,displayName

https://learn.microsoft.com/en-us/graph/onenote-best-practices

### Get all the sections
https://graph.microsoft.com/v1.0/me/onenote/notebooks?$expand=sections($select=id,self,displayName),sectionGroups($select=id,self,displayName)&$select=id,self,displayName

### Get all the pages in a section

Default ordering is modified by date, and it's faster when you sort by anything else.  So try:

GET ~/sections/{id}/pages?$select=id,title,createdDateTime&$orderby=createdDateTime
https://graph.microsoft.com/v1.0/me/onenote/sections/0-D837C5F832A7D046!11500/pages?$select=id,title,createdDateTime&$orderby=createdDateTime


https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/sections/0-D837C5F832A7D046!64051/pages?$select=id,self,title,contenturl


* [x] Why is it not listing all the pages? Maybe I need to involve section groups. No it was because you need to handle data chunking.

### Get the content of a page
https://graph.microsoft.com/v1.0/me/onenote
../pages/{page-id}/content
example page id:
    0-bfa9b56eccd54ae681ee7002a018ee2f!1-D837C5F832A7D046!11500

https://graph.microsoft.com/v1.0/me/onenote/pages/0-bfa9b56eccd54ae681ee7002a018ee2f!1-D837C5F832A7D046!11500/content

https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python

### Extracting text from a page

One Note page content is returned to as as html.  We'll use Beautiful Soup to extract the text.

* [Beautiful Soup docs](https://beautiful-soup-4.readthedocs.io/en/latest/#quick-start)
* [Quick and dirty Beautiful Soup example on Stack Exchange](https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python)

In [130]:
response = one_note._request_plain(page.content_url)
response.text

'<html lang="en-US">\r\n\t<head>\r\n\t\t<title>Game Mechanics/ideas</title>\r\n\t\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n\t\t<meta name="created" content="2022-09-23T15:48:00.0000000" />\r\n\t</head>\r\n\t<body data-absolute-enabled="true" style="font-family:Calibri;font-size:11pt">\r\n\t\t<div style="position:absolute;left:48px;top:115px;width:576px">\r\n\t\t\t<ul>\r\n\t\t\t\t<li>Ai dungeon multiplayer but each player has secret goals</li>\r\n\t\t\t\t<br />\r\n\t\t\t\t<li>A book that gives you goals</li>\r\n\t\t\t\t<li>You achieve the goals and get a currency</li>\r\n\t\t\t\t<li>A la dinkum</li>\r\n\t\t\t\t<li>Lots of sound effects and animations, like when you get currency it sounds like little clapping sounds</li>\r\n\t\t\t</ul>\r\n\t\t\t<br />\r\n\t\t\t<br />\r\n\t\t\t<ul>\r\n\t\t\t\t<li>When you&#39;re hungry, the world turns grey</li>\r\n\t\t\t\t<li>Rare things you can sell</li>\r\n\t\t\t\t<li>Everything is sellable</li>\r\n\t\t\t\t<li>Need a co

In [137]:
soup = BeautifulSoup(response.text, features='html.parser')
text = soup.get_text()
text

"\n\nGame Mechanics/ideas\n\n\n\n\n\n\nAi dungeon multiplayer but each player has secret goals\n\nA book that gives you goals\nYou achieve the goals and get a currency\nA la dinkum\nLots of sound effects and animations, like when you get currency it sounds like little clapping sounds\n\n\n\n\nWhen you're hungry, the world turns grey\nRare things you can sell\nEverything is sellable\nNeed a cool zoo/menagerie in the palace\nDisease- you gotta lay down on a couch and get leeched\nLittle dudes running around that fetch stuff for you and manage inventory\n\nBasically like a manual version of the conveyor system in space eng\n\n\n\n\n\n\n\n"

Let's put this in a text file so we can compare it side by side with the original OneNote.

In [141]:
with open('local/x.txt', 'w') as f:
    f.write(text)

This is ok, but certain context is lost by not preserving bullet points.  Let's try the `markdownify` library, and represent the HTML as markdown and see how that goes.

In [142]:
from markdownify import markdownify as md

In [145]:
md_text = md(response.text)
md_text

"\n\nGame Mechanics/ideas\n\n\n\n\n\n* Ai dungeon multiplayer but each player has secret goals\n  \n* A book that gives you goals\n* You achieve the goals and get a currency\n* A la dinkum\n* Lots of sound effects and animations, like when you get currency it sounds like little clapping sounds\n\n\n  \n\n  \n\n* When you're hungry, the world turns grey\n* Rare things you can sell\n* Everything is sellable\n* Need a cool zoo/menagerie in the palace\n* Disease- you gotta lay down on a couch and get leeched\n* Little dudes running around that fetch stuff for you and manage inventory\n\n\n\t+ Basically like a manual version of the conveyor system in space eng\n\n\n  \n\n\n\n\n"

In [164]:
with open('local/x.md', 'w') as f:
    f.write(md_text)

In [160]:
#page = one_note.find_page_from_path("Richard's Notebook", "Projects", "Interviewing")
page = one_note.find_page_from_path("Richard's Notebook", "Planning", "Bodybuilding Foods")

In [161]:
page

NotebookPageRef(name='Bodybuilding Foods', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-5bfbaecfa7754726a0c159bcad20f6eb!1-D837C5F832A7D046!203', content_url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/pages/0-5bfbaecfa7754726a0c159bcad20f6eb!1-D837C5F832A7D046!203/content')

In [162]:
page_html = one_note.get_page_html(page)

In [163]:
md_text = md(page_html.content)

In [170]:
one_note.get_page_md(page)

'\n\nBodybuilding Foods\n\n\n\n\n\n  \n\n<http://www.bodybuilding.com/fun/huge-on-a-hundred.html>\n\n\n  \n\nBodybuilding foods for $100\n\n\n  \n\nproteins\n--------\n\n\nChicken thighs 15 lbs\n\n\nJumbo eggs, 4 dozen (not large eggs. See if you can find them humane)\n\n\nred meat, such as london broil. Lean beef.\n\n\nCottage cheese. Lots of casein and low fat.\n\n\nStrawberry preserves\n\n\nPears apples or peaches for cottage cheese\n\n\n  \n\nFruits, Veggies\n---------------\n\n\nlike kale\n\n\nZucchini\n\n\nCabbage (fiber, vitamin c)\n\n\nBananas\n\n\nApples\n\n\n  \n\n  \n\nCarbs\n-----\n\n\nPotatoes\n\n\nOats\n\n\nRice\n\n\n  \n\n  \n\nMeal plan\n\n\nBreakfast: 6 scrambled eggs and oatmeal\n\n\n8-9oz red meat meal each day, he used London Broil\n\n\nEat a 16oz of cottage cheese before bed\n\n\n  \n\nCreatine 4x daily during loading phase\n\n\n  \n\n  \n\n  \n\n  \n\n  \n\n  \n\nContainers to hold chicken\n\n\nStock pot\n\n\npropane\n\n\n  \n\n  \n\nResearch\n--------\n\n\nHow ma